In [ ]:
%pip install norse --quiet
%pip install tonic --quiet

In [ ]:
import os
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
# import tonic
import norse

In [ ]:
from metavision_core.event_io.raw_reader import RawReader
from metavision_core.event_io.py_reader import EventDatReader

In [ ]:
input_path_raw = "./data"
from metavision_core.utils import get_sample

get_sample(input_path_raw, folder=".")
get_sample(input_path_dat, folder=".")